In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [2]:
# selenium을 사용해서 가상 크롬을 실행한다. => 사용중인 크롬 버전과 같은 버전의 드라이버를 os에 맞게 다운받는다.
# https://googlechromelabs.github.io/chrome-for-testing/#stable
# https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.109/win64/chromedriver-win64.zip => 다운로드
# 다운받은 크롬 드라이버를 워크스페이스에 복사해 넣고 사용한다.
option = Options()
option.add_argument('--window-size=1890,1030') # 가상 크롬의 창 크기를 지정한다.
driver = webdriver.Chrome('./chromedriver.exe', options=option) # 가상 크롬을 실행한다.
# 지정된 시간 동안 로딩 되기를 기다리며 대기한다. 로딩이 완료되면 즉시 다음 단계로 진행한다.
driver.implicitly_wait(5)
# get() 메소드로 가상 크롬에 크롤링할 타겟 사이트를 띄운다.
# driver.get('https://www.edmunds.com/')
driver.get('https://www.edmunds.com/hyundai/ioniq-6/2023/consumer-reviews/?pagesize=50')
time.sleep(0.5)
# 가상 크롬에 실행된 사이트의 html 정보를 얻어온다.
html = driver.page_source
# print(html)
soup = BeautifulSoup(html, 'html.parser')
# print(soup)

In [3]:
# 별점
stars = soup.findAll('span', {'class': 'rating-stars text-primary-darker mr-0_25'})
s = []
print(len(stars))
for star in stars:
    # print(star.text.split()[0])
    s.append(star.text.split()[0])
print(s)

11
['5', '5', '5', '5', '5', '5', '5', '5', '4', '5', '5']


In [4]:
# 타이틀
titles = soup.findAll('h3', {'class': 'heading-5 d-inline mb-0'})
t = []
print(len(titles))
# print(titles[0].text)
for title in titles:
    # print(title.text.strip('" '))
    t.append(title.text.strip('" '))
print(t)

11
['Love Ioniq 6', '700 miles review', 'I can’t believe we went electric', 'Never going back to gas - Ioniq 6 is a home run', 'Ioniq 6 - A Very Impressive EV From Hyundai', 'Quiet luxurious ride', 'Everyone thinks I bought a Porsche!', 'Great ride for a reasonable price', 'nice car', 'Amazing', 'EV Heaven']


In [5]:
# 작성일, 차종
dates = soup.findAll('div', {'class': 'small text-gray-dark mb-2'})
d = []
m = []
print(len(dates))
for date in dates:
    date = date.text.split(',')[1]
    # print(date.split())
    # print('{} {}-{}'.format(date.split()[0], date.split()[3], date.split()[4]))
    dateStr = '{} {}-{}'.format(date.split()[0], date.split()[3], date.split()[4])
    d.append(dateStr.split()[0])
    m.append(dateStr.split()[1])
print(d)
print(m)

11
['07/27/2023', '08/01/2023', '09/25/2023', '08/28/2023', '08/26/2023', '09/09/2023', '09/27/2023', '12/12/2023', '11/15/2023', '10/10/2023', '11/08/2023']
['IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6', 'IONIQ-6']


In [6]:
# 리뷰
reviews = soup.findAll('div', {'class': 'truncated-text size-16'})
r = []
print(len(reviews))
for review in reviews:
    # print(review.text.split('Safety')[0])
    # print('=' * 80)
    r.append(review.text.split('Safety')[0])
print(r)

11
['The car has a very comfortable and quiet ride. There are a lot of security features and ways to adjust to your preference. Inside features are very nice and not overstated. The outside is more appealing, at least to me, in comparison to some other EV’s in the same price range. In addition, more car features available on the Limited version than available on other similarly priced EV’s in the current market. Lots of leg room in both the front and back seats. It is low to the ground so feels and handles like a sports car. ', 'Love the car; the sports version is zippy, the economy version gives 310+ miles range with AC on. Fast charging is free for two years\nThe interiors are awesome with the ambient lighting really giving a cool vibe. \nThe HDA, lane change and managing the lanes works mostly. This can be slightly better especially when changing lanes quickly\nI test drove: Model 3, ID.4, Bolt SUV premier, Model Y. Liked this car the best as a combination of build quality + drive q

In [7]:
# Safety, Technology, Performance, Interior, Comfort, Reliability, Value
grades = soup.findAll('div', {'justify-content-between flex-column flex-md-row row'})
print(len(grades))
gradeList = []

for grade in grades:
    gradeDict = {}
    # print(grade)
    temp = []
    gradeNames = grade.findAll('dt', {'class': 'font-weight-normal'})
    for gradeName in gradeNames:
        # print(gradeName.text)
        temp.append(gradeName.text)
    # print(temp)
    gradeStars = grade.findAll('span', {'class': 'rating-stars text-primary-darker'})
    for i in range(len(gradeStars)):
        # print(gradeStars[i].text[0])
        gradeDict[temp[i]] = gradeStars[i].text[0]
    # print(gradeDict)
    gradeList.append(gradeDict)
    # print('=' * 80)

11


In [8]:
safety = [0 for i in range(len(grades))]
technology = [0 for i in range(len(grades))]
performance = [0 for i in range(len(grades))]
interior = [0 for i in range(len(grades))]
comfort = [0 for i in range(len(grades))]
reliability = [0 for i in range(len(grades))]
value = [0 for i in range(len(grades))]

# for g in gradeList:
for i in range(len(gradeList)):
    for key in gradeList[i].keys():
        if key == 'Safety':
            safety[i] = int(gradeList[i]['Safety'])
        if key == 'Technology':
            technology[i] = int(gradeList[i]['Technology'])
        if key == 'Performance':
            performance[i] = int(gradeList[i]['Performance'])
        if key == 'Interior':
            interior[i] = int(gradeList[i]['Interior'])
        if key == 'Comfort':
            comfort[i] = int(gradeList[i]['Comfort'])
        if key == 'Reliability':
            reliability[i] = int(gradeList[i]['Reliability'])
        if key == 'Value':
            value[i] = int(gradeList[i]['Value'])
            
print('Safety:', safety)
print('Technology:', technology)
print('Performance:', performance)
print('Interior:', interior)
print('Comfort:', comfort)
print('Reliability:', reliability)
print('Value:', value)

Safety: [5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5]
Technology: [5, 4, 5, 5, 5, 5, 5, 5, 4, 5, 5]
Performance: [5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5]
Interior: [5, 5, 5, 5, 4, 5, 5, 4, 5, 5, 5]
Comfort: [5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 5]
Reliability: [5, 5, 5, 5, 5, 5, 5, 4, 5, 5, 5]
Value: [5, 4, 5, 5, 4, 5, 5, 5, 5, 5, 5]


In [9]:
df = pd.DataFrame({
    'star': s,
    'title': t,
    'date': d,
    'model': m,
    'review': r,
    'safety': safety,
    'technology': technology,
    'performance': performance,
    'interior': interior,
    'comfort': comfort,
    'reliability': reliability,
    'value': value
})
df

,star,title,date,model,review,safety,technology,performance,interior,comfort,reliability,value
0,5,Love Ioniq 6,07/27/2023,IONIQ-6,The car has a very comfortable and quiet ride....,5,5,5,5,5,5,5
1,5,700 miles review,08/01/2023,IONIQ-6,"Love the car; the sports version is zippy, the...",5,4,5,5,5,5,4
2,5,I can’t believe we went electric,09/25/2023,IONIQ-6,I continually told my wife that we did not nee...,5,5,5,5,5,5,5
3,5,Never going back to gas - Ioniq 6 is a home run,08/28/2023,IONIQ-6,"5 days in and I’m blown away with this car, it...",5,5,5,5,5,5,5
4,5,Ioniq 6 - A Very Impressive EV From Hyundai,08/26/2023,IONIQ-6,"Just for background, my last car was a Honda C...",5,5,5,4,5,5,4
5,5,Quiet luxurious ride,09/09/2023,IONIQ-6,"Ioniq 6 has the power, beauty and luxury feel ...",5,5,5,5,5,5,5
6,5,Everyone thinks I bought a Porsche!,09/27/2023,IONIQ-6,My first all-electric. Let me start out by say...,5,5,5,5,5,5,5
7,5,Great ride for a reasonable price,12/12/2023,IONIQ-6,I’ve had the car for about three weeks and lov...,5,5,5,4,5,4,5
8,4,nice car,11/15/2023,IONIQ-6,"confortable and sleek lines, my concern is tha...",0,4,4,5,4,5,5
9,5,Amazing,10/10/2023,IONIQ-6,This car is amazing. Everything works as descr...,5,5,5,5,5,5,5
